In [1]:
from flair.data import Sentence
from flair.models import SequenceTagger

from myMongoClient import MyMongoClient, BANQUE_DOCUMENT_FOLDER
%reload_ext autoreload
%autoreload 2

2024-02-27 16:00:35.108185: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-27 16:00:35.724879: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
db = "documents-100039"
organisation_id = int(db[-6:])

mongo_client = MyMongoClient(db)

In [3]:
import spacy


nlp_spacy = spacy.load("en_core_web_sm")

# Function to extract entities from a given text using SpaCy
def extract_entities_spacy(text, split_paragraph):
    texts = text.split('\n') if split_paragraph else [text]
    entities = []
    for text in texts :
        if text :
            doc = nlp_spacy(text)
            entities.extend([(ent.text, ent.label_) for ent in doc.ents])
    return entities


tagger_flair_ontonote = SequenceTagger.load("flair/ner-english-ontonotes")

# Function to extract entities from a given text using Flair
def extract_entities_flair(text, tagger, split_paragraph = False):
    texts = text.split('\n') if split_paragraph else [text]
    entities = []
    for text in texts :
        if text :
            sentence = Sentence(text)
            tagger.predict(sentence)
            entities.extend([(entity.text, entity.get_labels()[0].value) for entity in sentence.get_spans('ner')])
    return entities

/home/ketl2/.local/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


2024-02-27 16:00:44,209 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC


In [4]:
from myMongoClient import MyMongoClient, load, dump
gold_labels = load("data/labels_results/2024-02-02-gold_labels_filenames_clean.pkl")

In [5]:
def dump_documents(path) :
    documents = {}
    for doc in mongo_client.get_all_documents(BANQUE_DOCUMENT_FOLDER):
        doc_id = doc['_id']
        doc_content = mongo_client.get_document_content(document_id=doc_id)
        # entities = extract_entities_flair(doc_content, tagger_flair_ontonote, split_paragraph= True)
        entities = extract_entities_spacy(doc_content, split_paragraph= True)
        documents[doc_id] = {'doc_content' : doc_content,
                              'entities' : entities, 
                            #   'entities_full_doc' : extract_entities_spacy(doc_content, tagger_flair_ontonote, split_paragraph= False),
                              'entities_full_doc' : extract_entities_spacy(doc_content, split_paragraph= False),
                              'gold_labels' : gold_labels['labels'][doc_id] if doc_id in gold_labels['labels'] else {}}
    dump(documents, path)

In [7]:
path =  "data/labels_results/documents_spacy2_entities.pkl"
dump_documents(path)
documents = load(path)

In [8]:
documents

{'LrbMyOW9+ohDSwkCB3juHw==': {'doc_content': 'HG\n                                  Hôpitaux\n                                  Universitaires\n                                  Genève\nDirection des ressources humaines HUG\nRue Gabrielle-Perret-Gentil 4\nCH - 1211 Genève 14\nPour tout renseignement: tél. 022.372.63.04                                                          Monsieur\n                             fax.                                                                    NICOLAS PETIT\n                                                                                                     Rue Centrale 92\n                                                                                                     1003 Lausanne\nBULLETIN DE SALAIRE\nmars 2021\nFonction(s) : MEDECIN ADJOINT.E AGREGE.E\nIdentité : 612787        % Act. : 100.00       Cl./An. : 27/15\nGE\nCD                                        Rubriques                                                 HE Base DOG        T

In [9]:
from datetime import datetime 
def is_valid_date(date_string):
    try:
        # Attempt to parse the date string
        date = datetime.fromisoformat(date_string).strftime('%Y-%m-%d')
        return date
    except ValueError:
        # If the parsing fails, it's not a valid date
        return False
    
def is_float(value):
    try:
        float_string = value.replace("'", "")
        float_string = float_string.replace(",", ".")
        float_value = float(float_string)
        return float_value
    except ValueError:
        return False
    
def is_int(value):
    try:
        int_string = value.replace("'", "")
        int_value = int(int_string)
        return int_value
    except ValueError:
        return False

def parse_values(value) :
    date = is_valid_date(value)
    if date :
        return date

    int_val = is_int(value)
    if int_val :
        return int_val
    
    float_val = is_float(value)
    value = float_val if float_val else value

    return value

nb_value_checked, value_found_split, value_found_doc = {},{}, {}
for doc_id, doc in documents.items():
    entities = set([parse_values(ent) for ent, tag in doc['entities']])
    entities_full_doc = set([parse_values(ent) for ent, tag in doc['entities_full_doc']])
    gold_values = [(gold_label,parse_values(gold_value)) for gold_label, gold_value in doc['gold_labels'].items() 
                   if gold_label not in ['language', 'client', 'document type','immeuble']]

    for gold_label, gold_value in gold_values :
        value_split, value_doc = int(sum([str(gold_value) in str(entity) for entity in  entities])) >0 , int(sum([str(gold_value) in str(entity) for entity in entities_full_doc])) >0 
        value_found_split[gold_label] = value_found_split[gold_label] + value_split if gold_label in value_found_split else value_split
        value_found_doc[gold_label] = value_found_doc[gold_label] + value_doc if gold_label in value_found_doc else value_doc
        nb_value_checked[gold_label] = nb_value_checked[gold_label] + 1 if gold_label in nb_value_checked else 1

score_by_field_split = {key : val/nb_value_checked[key] for key, val in value_found_split.items()}
score_by_field_doc = {key : val/nb_value_checked[key] for key, val in value_found_doc.items()}
score_split = sum(value_found_split.values())/sum(nb_value_checked.values())
score_doc = sum(value_found_doc.values())/sum(nb_value_checked.values())

score_split,score_doc

(0.2815533980582524, 0.2912621359223301)

In [10]:
sum([False, True, False])

1